# Создание решения по стандартизации названий спортивных школ.

**Описание проекта**

<br>Сервис "Мой Чемпион" помогает спортивным школам фигурного катания, тренерам
<br>мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель**

- Создать решение для стандартизации названий спортивных школ.
  <br>Например, одна и та же школа может быть записана по-разному
  <br>Необхдодимо сопоставить эти варианты эталонному названию из предоставленной таблицы

**Задачи**

- Изучить данные – эталонные названия СШ и варианты пользовательского ввода
- Подготовить обучающий набор данных на основе эталонного датасета
- Создать модель для подбора наиболее вероятных названий при ошибочном вводе
- Создать функцию (класс, модуль) для применения в сервисе
  - возможность выбора количества кандидатов
  - вывод в виде списка словарей
- Протестировать решение
- Проанализировать результат и предложить варианты улучшения
- Создать документацию
  - описание признаков
  - какая модель используется
  - как оценивается качество
  - инструкция по запуску (применению)
- Создать демо приложение

**План выполнения**

- Загрузка данных;
- Объединение наборов данных;
- Исследовательский анализ данных;
- Чистка данных;
- Получение эмбеддингов;
- Обучение модели;
- Проверка работоспособности;
- Заключение.

## Установка зависимостей

In [13]:
# !pip install -U sentence-transformers
# !pip install pandas

!pip freeze > requirements.txt

In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict
from sentence_transformers import SentenceTransformer, util

/home/denis/Documents/main_venv/goprotect_2/gp.env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Объявление глобальных переменных

In [2]:
DATA_DIR = 'data'
MODEL_DIR = 'services/models'
RANDOM_STATE = 42
TEST_SIZE = 0.1

pd.options.mode.copy_on_write = True

Пользовательские функции

In [84]:
def school_name_search(input_example: str, embeddings: np.array, model: SentenceTransformer(), k: int = 5) -> Dict:
    res = util.semantic_search(model.encode(input_example), name_embeddings, top_k=k)

    idx = [i['corpus_id'] for i in res[0]]
    score = [i['score'] for i in res[0]]
    valid_idx = [i for i in idx if i in df.index]  # Filter idx based on existing df index
    if not valid_idx:
        return None  # Handle no matches (optional)
    result = (df.loc[idx, ['school_id', 'standard']]
              .assign(cosine_similarity=score)
              .drop_duplicates(subset=['school_id'])
              .iloc[:k].rename(columns={'standard': 'school'}))
    return result.to_dict(orient='records')

def create_result_column(df: pd.DataFrame, name_embeddings: np.array, model: SentenceTransformer(), column_name: str = 'labse_default') -> pd.DataFrame:
    school_names = df['name'].tolist()

    name_embeddings = []
    with tqdm(school_names, desc="Getting results") as progress_bar:
      for name in progress_bar:
        res_name = school_name_search(name, name_embeddings, model, k=1)
        name_embeddings.append(res_name[0]['school'])

    df[column_name] = name_embeddings

    return df

## Знакомство с данными

In [3]:
schools = pd.read_csv(os.path.join(DATA_DIR, 'Школы.csv'))
schools.tail()

,school_id,name,region
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край
305,1836,"ООО ""Триумф""",Москва


In [4]:
sample = pd.read_csv(os.path.join(DATA_DIR, 'Примерное написание.csv'))
sample.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


Выводы и наблюдения:
- оба датасета загружены;
- наблюдается общий признак для объединения;
- одной записи в эталонном датасете может соответствовать несколько записей во втором.

## Объединение датасетов

In [5]:
df = sample.merge(schools, how='left', on='school_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name_x     895 non-null    object
 2   name_y     895 non-null    object
 3   region     895 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.1+ KB


In [6]:
df.rename(columns={'name_x': 'name', 'name_y': 'target'}, inplace=True)
df['standard'] = df['target'] + ': ' + df['region']
df.drop(columns=['target', 'region'], axis=1, inplace=True)
df.tail(10)

,school_id,name,standard
885,5,Айсдрим,Ice Dream / Айс Дрим: Санкт-Петербург
886,4,КФК Аврора,Аврора: Санкт-Петербург
887,4,"МО г.Петергоф, СФК «Аврора»",Аврора: Санкт-Петербург
888,3,"Республика Татарстан, МБУ СШОР ""ФСО ""Авиатор""",Авиатор: Республика Татарстан
889,3,"Республика Татарстан, МБУ СШОР ""ФСО ""Авиатор""""...",Авиатор: Республика Татарстан
890,3,"Республика Татарстан, СШОР ФСО Авиатор",Авиатор: Республика Татарстан
891,3,"СШОР ФСО Авиатор, Республика Татарстан",Авиатор: Республика Татарстан
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",Авиатор: Республика Татарстан
893,2,"ЯНАО, СШ ""Авангард""",Авангард: Ямало-Ненецкий АО
894,1,"Московская область, СШ ""Авангард""",Авангард: Московская область


Выводы и наблюдения:
- эталонный набор данных присоединён ко второму;
- произведено объединение названия школы с регионом.

## Исследовательский анализ данных

In [7]:
print(f"Количество полных дубликатов строк: {df.duplicated().sum()}")
print(f"Количество уникальных значений целевой переменной: {df['standard'].nunique()}")
print(f"Количество дубликатов строк по признаку 'name': {df['name'].duplicated().sum()}")
print('Совпадения строк в признаке name:')
display(df[df.duplicated(subset='name', keep=False)])
df = df[df['school_id'] != 277]
print('Удалена строка с повторяющимся name, но разным standard')
df.info()

Количество полных дубликатов строк: 0
Количество уникальных значений целевой переменной: 264
Количество дубликатов строк по признаку 'name': 1
Совпадения строк в признаке name:


,school_id,name,standard
34,277,"КФК ""Динамо-Санкт-Петербург""","НП КФК ""Динамо-Санкт-Петербург"": Санкт-Петербург"
758,48,"КФК ""Динамо-Санкт-Петербург""",Динамо Санкт-Петербург: Санкт-Петербург


Удалена строка с повторяющимся name, но разным standard
<class 'pandas.core.frame.DataFrame'>
Index: 893 entries, 0 to 894
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  893 non-null    int64 
 1   name       893 non-null    object
 2   standard   893 non-null    object
dtypes: int64(1), object(2)
memory usage: 27.9+ KB


In [8]:
print(f"В среднем на одно корректное наименование приходится {round(df['name'].nunique() / df['standard'].nunique(),)} некорректных")

В среднем на одно корректное наименование приходится 3 некорректных


Выводы и наблюдения:
- полных повторов в данных нет;
- пропусков в данных нет;
- удалена одна строка с повторяющимся значением в признаке `name`.

## Разработка моделей машинного обучения

### Baseline

In [9]:
try:
    labse = joblib.load(os.path.join(MODEL_DIR, 'labse.model'))
except:
    labse = SentenceTransformer('sentence-transformers/LaBSE')
    joblib.dump(labse, os.path.join(MODEL_DIR, 'labse.model'))
labse

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
)

Получение эмбеддингов наименований школ

In [10]:
school_names = df['name'].values

# Wrap the list of school names with tqdm for progress bar
with tqdm(school_names, desc="Generating Name Embeddings") as progress_bar:
  name_embeddings = []
  for name in progress_bar:
    embedding = labse.encode(name)
    name_embeddings.append(embedding)

name_embeddings = np.array(name_embeddings)  # Convert list to numpy array
name_embeddings.shape

Generating Name Embeddings: 100%|██████████████████████████████████████| 893/893 [02:06<00:00,  7.05it/s]


(893, 768)

In [23]:
df.reset_index(drop=True, inplace=True)
df = create_result_column(df.copy(), name_embeddings, labse)

Getting results: 100%|█████████████████████████████████████████████████| 893/893 [02:16<00:00,  6.54it/s]


Расчёт точности

In [61]:
accuracy_labse_default = (df['labse_default'] == df['standard']).mean()
print("Baseline accuracy:", round(accuracy_labse_default, 2))

Baseline accuracy: 1.0


Проверка работоспособности

In [60]:
school_name_search('школа Бережной', name_embeddings, labse, k=5)

[{'school_id': 24,
  'school': 'Школа ФК Е. Бережной: Санкт-Петербург',
  'cosine_similarity': 0.5782665610313416},
 {'school_id': 128,
  'school': 'Орленок: Пермский край',
  'cosine_similarity': 0.5124238729476929},
 {'school_id': 302,
  'school': 'СШ №2: Республика Башкортостан',
  'cosine_similarity': 0.5083849430084229},
 {'school_id': 610,
  'school': 'СШОР "Надежда Губернии: Саратовская область',
  'cosine_similarity': 0.49365147948265076}]

Выводы и заключения:
- в качестве baseline принята мультиязычная модель на основе BERT LaBSE;
- при выводе нескольких результатов только первый показывает релевантное наименование;
- в среднем при проверке по всему набору данных показатель accuracy получился очень высоким.

### MiniLM-L12

In [63]:
try:
    minilm = joblib.load(os.path.join(MODEL_DIR, 'minilm.model'))
except:
    minilm = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
    joblib.dump(minilm, os.path.join(MODEL_DIR, 'minilm.model'))
minilm

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [64]:
school_names = df['name'].values

# Wrap the list of school names with tqdm for progress bar
with tqdm(school_names, desc="Generating Name Embeddings") as progress_bar:
  name_embeddings = []
  for name in progress_bar:
    embedding = minilm.encode(name)
    name_embeddings.append(embedding)

name_embeddings = np.array(name_embeddings)  # Convert list to numpy array
name_embeddings.shape

Generating Name Embeddings: 100%|██████████████████████████████████████| 893/893 [00:55<00:00, 16.01it/s]


(893, 384)

In [85]:
df_mlm = create_result_column(df = df.copy(),
                              name_embeddings = name_embeddings,
                              model = minilm,
                              column_name = 'minilm_default')
accuracy_minilm_default = (df_mlm['minilm_default'] == df_mlm['standard']).mean()
print("MiniLM accuracy:", round(accuracy_minilm_default, 2))

Getting results: 100%|█████████████████████████████████████████████████| 893/893 [00:58<00:00, 15.23it/s]

MiniLM accuracy: 1.0


In [88]:
school_name_search('авангард', name_embeddings, minilm, k=5)

[{'school_id': 2,
  'school': 'Авангард: Ямало-Ненецкий АО',
  'cosine_similarity': 0.8560034036636353},
 {'school_id': 1,
  'school': 'Авангард: Московская область',
  'cosine_similarity': 0.7600955367088318},
 {'school_id': 5,
  'school': 'Ice Dream / Айс Дрим: Санкт-Петербург',
  'cosine_similarity': 0.7112890481948853},
 {'school_id': 62,
  'school': 'Звездный лед: Санкт-Петербург',
  'cosine_similarity': 0.6321210861206055}]

Выводы и заключения:
- в качестве второй модели принята лёгкая мультиязычная модель на основе `BERT` `all-MiniLM-L12-v2`;
- вывод нескольких результатов больше соответствует действительности;
- показатель косинуснуй близости в среднем гораздо выше, чем у `baseline`;
- в среднем при проверке по всему набору данных показатель `accuracy` получился очень высоким.

## Заключение

Отчёт о проделанной работе
1. Предоставленные Заказчиком данные загружены, изучены, почищены
   - серьёзных проблем не обнаружено;
   - дубликаты удалены;
   - произведено объединение наборов данных;
2. получены эмбеддинги и произведён расчёт косинусного сходства для двух моделей:
   - `sentence-transformers/LaBSE`;
   - `sentence-transformers/all-MiniLM-L12-v2`;
3. посчитана `accuracy`, произведена выборочная ручная проверка результатов;
   - обе модели показали отличный результат на всём датасете по метрике;
   - выборочная проверка указала на то, что вторая модель предлагает более корректные варианты,
     <br>когда требуется более одного вывода;
   - вторая модель занимает на порядок меньший размер дискового пространства.

<br>**Общий вывод: для решения поставленной задачи хорошо подходит модель `sentence-transformers/all-MiniLM-L12-v2`**